In [1]:
import os
from dotenv import load_dotenv
import openai
# Load environment variables from the .env file
load_dotenv()
# Access environment variables
api_key = os.getenv('API_KEY')
api_type = os.getenv('OPENAI_API_TYPE')
model_name = os.getenv('MODEL_NAME')

#base_url = os.getenv('BASE_URL')
azure_endpoint = os.getenv('BASE_URL')
openai_api_base=azure_endpoint
#DM_db_url = os.getenv('DRUGMATRIX_DB_URL')
#database_url = os.getenv('DATABASE_URL')
debug_mode = os.getenv('DEBUG')
engin = os.getenv('MODEL_ENGINE')
version = os.getenv('OPENAI_API_VERSION')


os.environ["OPENAI_API_TYPE"] = api_type
os.environ["OPENAI_API_BASE"] = os.getenv('BASE_URL')
os.environ["OPENAI_API_KEY"] =  api_key
#os.environ["AZURE_OPENAI_ENDPOINT"] = base_url
#os.environ["AZURE_ENDPOINT"] = base_url 
os.environ["OPENAI_API_VERSION"] = version # "2023-05-15"
openai.api_type = 'azure'
AZURE_OPENAI_API_VERSION="2024-02-01"
openai.api_version = "2024-02-01"
AZURE_OPENAI_ENDPOINT=os.getenv('BASE_URL')

print(f"base url: {azure_endpoint}")
print(f"API_KEY : {api_key}")
print(f"engin: {engin}")
print(f"model_name: {model_name}")
print(f"api_type: {api_type}")
print(f"version : {version }")


base url: https://litellm.toxpipe.niehs.nih.gov
API_KEY : sk-dSTROKbqfoD8L2WDNDhwXA
engin: azure-gpt-4o
model_name: azure-gpt-4
api_type: azure
version : 2023-05-15


In [9]:
 
#from llama_index.llms import AzureOpenAI
from langchain.llms import AzureOpenAI
#from llama_index.embeddings import AzureOpenAIEmbedding
from langchain.embeddings import AzureOpenAIEmbeddings
                                 

llm = AzureOpenAI(
    #engine="azure-gpt-4o", # model_name on litellm proxy
    #model_name="azure-gpt-4o",
    deployment="gpt-4",   
    model_name=model_name,
    temperature=0.0,
    #azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)

embed_model = AzureOpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    #azure_endpoint="http://litellm.toxpipe.niehs.nih.gov",
    #azure_endpoint=azure_endpoint,
    openai_api_key=api_key,
    #openai_api_key="sk-dSTROKbqfoD8L2WDNDhwXA",
    openai_api_version="2024-02-01",
    chunk_size=100,
    validate_base_url=True,
    
)


/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! deployment is not default parameter.
                deployment was transferred to model_kwargs.
                Please confirm that deployment is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)
/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/langchain_community/llms/openai.py:882: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://litellm.toxpipe.niehs.nih.gov to https://litellm.toxpipe.niehs.nih.gov/openai.
  warnings.warn(
/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:114: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_u

In [10]:
from llama_index.core import ( VectorStoreIndex, SimpleDirectoryReader, StorageContext,load_index_from_storage,)


In [11]:
from llama_index.core import ServiceContext

In [13]:
documents = SimpleDirectoryReader("./test_data/").load_data()
print(len(documents))
from llama_index.core import Document, VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

58


In [15]:
# Use MultiModal Retriever and Query Engine
from llama_index.core import PromptTemplate
from llama_index.core.query_engine import SimpleMultiModalQueryEngine
#return top answer
retriever_engine = index.as_retriever(similarity_top_k=1, image_similarity_top_k=3)


In [16]:
retrieval_results2 = retriever_engine.retrieve("When is the Joe Wong 中文脱口秀北卡 show")
for result in retrieval_results2:
    print (result.text)

Order Information
Do you organize events?
Start selling in minutes with Eventbrite!
www.eventbrite.com黄西 Joe Wong 中文脱口秀北卡 RTP专场
《水土不服》
Sunday, November 10, 2024 from 5:30 PM to 6:30 PM (EST)Gross Hall, 1 Towerview Road, Durham, NC 27710General早鸟票  $44.52
Order #10749729879. Ordered by Sue Nolte on October
15, 2024 8:53 AMEventbrite Completed
1074972987917724837359001
Order #10749729879


In [12]:
##migration to newer version
documents = SimpleDirectoryReader("./test_data/").load_data()
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()
response = query_engine.query("Can you list the models of porsche?")
print(response)

ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/

In [10]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

#Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.llm = llm
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

In [11]:

documents = SimpleDirectoryReader("./test_data/").load_data()
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model#, transformations=transformations
)


In [13]:
print(index)

In [12]:
# ... until you create a query engine
#query_engine = index.as_query_engine()
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("Can you list the models of porsche?")
print(response)

/ddn/gs1/home/noltesz/.conda/envs/py311/lib/python3.11/site-packages/llama_index/llms/langchain/base.py:106: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_str = self._llm.predict(prompt, **kwargs)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [66]:
## Meta AI 
## Error API key 

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    documents = SimpleDirectoryReader("./test_data/").load_data()
    #index = VectorStoreIndex.from_index_storage_context.persist(documents)
    index = VectorStoreIndex.from_documents(documents)
else:
    storage_context = StorageConte(PERSIST_DIR)
    index = load_index_from_storag(storage_context)
        
query_engine = index.as_query_engine()
while True:
    user_input=input(">>>: ")
    response = query_engine.query (user_input)
    print (response)
    
    
 

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-dSTRO*************hwXA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [73]:
## chat GPT

import os
from llama_index.core import VectorStoreIndex, StorageContext, SimpleDirectoryReader, load_index_from_storage
 
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    os.makedirs(PERSIST_DIR)
    documents = SimpleDirectoryReader("./test_data/").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(PERSIST_DIR)
else:
    storage_context = StorageContext.from_persist_dir(PERSIST_DIR)
    index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()
while True:
    user_input = input(">>>: ")
    response = query_engine.query(user_input)
    print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-dSTRO*************hwXA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [67]:
###chatgpt
import os
from llama_index import VectorStoreIndex, StorageContext, SimpleDirectoryReader, load_index_from_storage

PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    os.makedirs(PERSIST_DIR)
    documents = SimpleDirectoryReader("./test_data/").load_data()
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(PERSIST_DIR)
else:
    storage_context = StorageContext.from_persist_dir(PERSIST_DIR)
    index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()
while True:
    user_input = input(">>>: ")
    response = query_engine.query(user_input)
    print(response)

ImportError: cannot import name 'VectorStoreIndex' from 'llama_index' (unknown location)

In [59]:
print(VectorStoreIndex)

<class 'llama_index.core.indices.vector_store.base.VectorStoreIndex'>


In [12]:
from langchain.document_loaders import PyPDFLoader
#loader = PyPDFLoader("./test_data/my-ticket.pdf")
#pages = loader.load_and_split()

def load_pdf_files2(directory):
    """Load PDF file contents from a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            loader = PyPDFLoader(file_path)           
            pages = loader.load_and_split()
            documents.append(pages)
    return documents

In [17]:
from langchain.vectorstores.faiss import FAISS

In [18]:
from langchain.document_loaders import PyPDFLoader
#loader = PyPDFLoader("./test_data/my-ticket.pdf")
#pages = loader.load_and_split()

def load_pdf_files2(directory):
    """Load PDF file contents from a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            loader = PyPDFLoader(file_path)           
            pages = loader.load_and_split()
            documents.append(pages)
    return documents



In [20]:
# Load PDF files
directory = './test_data'
documents = load_pdf_files2(directory)

# Invoke vectorstore
#vectorstore(documents)

In [23]:
#print(documents)

In [24]:
#text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
#docs = text_splitter.split_documents(documents)

#print(docs)



In [ ]:
chain = load_qa_chain(llm_openAI, chain_type="refine")

## Sue env test

In [11]:
import os
from openai import AzureOpenAI
client = AzureOpenAI(
    api_key="3b683e793d194186876e6508296182ca",
    #azure_endpoint="https://nih-niehs-toxpipe-sue-test.openai.azure.com/",
    azure_endpoint="https://nih-niehs-toxpipe-sue-test.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-08-01-preview",
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=version,
    #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
 
response = client.chat.completions.create(
    model="gpt-4", # model to send to the proxy
    messages = [
        {
            "role": "user",
            "content": "this is a test request, when is the U.S. election day?"
        }
    ]
)
 
#print(response)
print(response.choices[0].message.content)

In the United States, Election Day is held on the first Tuesday after the first Monday in November. In 2022, it will be on November 8th.


In [44]:
import os
import requests
import base64

# Configuration
API_KEY = "3b683e793d194186876e6508296182ca"
IMAGE_PATH = "./imgs/user.png"
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "Can you describe the image?",
          "text": "You are an AI assistant that helps people find information."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}
 
#ENDPOINT = "https://nih-niehs-toxpipe-sue-test.openai.azure.com/openai/deployments/gpt-4/chat/completions?api-version=2024-02-15-preview"
ENDPOINT = "https://nih-niehs-toxpipe-sue-test.openai.azure.com/"

# Send request
try:
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

SystemExit: Failed to make the request. Error: 404 Client Error: Resource Not Found for url: https://nih-niehs-toxpipe-sue-test.openai.azure.com/

In [ ]:
sue_llm = AzureOpenAI(
    #engine="azure-gpt-4o", # model_name on litellm proxy
    model_name="azure-gpt-4o",
    temperature=0.0,
    azure_endpoint="https://litellm.toxpipe.niehs.nih.gov", # litellm proxy endpoint
    api_key="sk-dSTROKbqfoD8L2WDNDhwXA", # litellm proxy API Key
    api_version="2024-02-01",
)

In [4]:
from openai import AzureOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
#from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core  import VectorStoreIndex, StorageContext, SimpleDirectoryReader, load_index_from_storage

#print(f"api key: {api_key}")
llm = AzureOpenAI(
    #engine="azure-gpt-4o", # model_name on litellm proxy
    #temperature=0.0,
    azure_endpoint=azure_endpoint,
    api_key=api_key, # litellm proxy API Key
    api_version=version,
)
 

In [5]:
embed_model = AzureOpenAIEmbeddings(
    deployment="text-embedding-ada-002",
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=version,
    chunk_size=100,
    validate_base_url=True,
)


ValidationError: 2 validation errors for AzureOpenAIEmbeddings
openai_api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='3b683e793d194186876e6508296182ca', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden
openai_api_version
  Extra inputs are not permitted [type=extra_forbidden, input_value='2024-07-01-preview', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

In [ ]:
llm = AzureOpenAI(
    engine="azure-gpt-4o", # model_name on litellm proxy
    temperature=0.0,
    azure_endpoint=azure_endpoint,
    api_key=api_key, # litellm proxy API Key
    api_version=version,
)

embed_model = AzureOpenAIEmbedding(
    deployment_name="text-embedding-ada-002",
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=version,
)

documents = SimpleDirectoryReader("llama_index_data").load_data()
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)